In [33]:
import json
from tqdm import tqdm

from openai import OpenAI
from elasticsearch import Elasticsearch

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [32]:
documents[0]

{'text': 'Intervalltraining ist eine Trainingsmethode, bei der sich intensive Belastungsphasen mit Erholungsphasen abwechseln. Durch diese Wechsel zwischen hoher und niedriger Belastung werden Ausdauer und Schnelligkeit effektiv verbessert, da der Körper sowohl anaerob als auch aerob gefordert wird.',
 'section': 'Training',
 'question': 'Was ist Intervalltraining?',
 'course': 'running-assistant-rag'}

In [12]:
es_client = Elasticsearch('http://localhost:9200')

In [23]:
es_client.info()

ObjectApiResponse({'name': 'e51f4087254f', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'jUUibplUT_6lB6igj22f6w', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [24]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "running-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'running-questions'})

In [34]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████| 98/98 [00:00<00:00, 264.56it/s]


In [37]:
query = 'Was ist Intervalltraining?'

In [39]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "running-assistant-rag"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [40]:
elastic_search(query)

[{'text': 'Intervalltraining ist eine Trainingsmethode, bei der sich intensive Belastungsphasen mit Erholungsphasen abwechseln. Durch diese Wechsel zwischen hoher und niedriger Belastung werden Ausdauer und Schnelligkeit effektiv verbessert, da der Körper sowohl anaerob als auch aerob gefordert wird.',
  'section': 'Training',
  'question': 'Was ist Intervalltraining?',
  'course': 'running-assistant-rag'},
 {'text': 'Intervalltraining ist eine Trainingsmethode, bei der sich intensive Belastungsphasen mit Erholungsphasen abwechseln. Durch diese Wechsel zwischen hoher und niedriger Belastung werden Ausdauer und Schnelligkeit effektiv verbessert, da der Körper sowohl anaerob als auch aerob gefordert wird.',
  'section': 'Training',
  'question': 'Was ist Intervalltraining?',
  'course': 'running-assistant-rag'},
 {'text': 'Intervalltraining ist eine Trainingsmethode, bei der sich intensive Belastungsphasen mit Erholungsphasen abwechseln. Durch diese Wechsel zwischen hoher und niedriger B

In [14]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [18]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [19]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi4-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [43]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [47]:
rag('Was ist Intervalltrainig?')

'Intervalltraining ist eine Trainingstechnik, die Perioden intensiver Aktivitätsaufbauphase (Stressarbeit), gekennzeichnet durch hohe Erholungsfristen oder lange Ruhezeiten kombiniert mit Phasen niedriger Aktivität und kurzen Erholungsphasen enthält. Diese Abwechselung zwischen hohen Belastungen (z.B. Sprinten) und niedrigeren (Leichtkardio, Training bei konstanter Intensität) ermöglicht es dem Körper, sowohl anaerob als auch aerob zu trainieren, was die Ausdauer- und Schnelligkeitsentwicklung effektiver fördert als eine kontinuierliche intensivere Aktivität ohne regelmäßige Entspannungsphasen.'

In [49]:
# llm('Was ist Intervalltrainig?')